In [6]:
import numpy as np
import time 
import matplotlib.pyplot as plt
from util import *

%load_ext autoreload
%autoreload 2

In [19]:
from keras.layers import Input, Convolution2D, Flatten, Dense, Activation
from keras.models import Sequential
from keras.optimizers import SGD , Adam
from keras.utils import plot_model
from keras import regularizers
# apply a 3x3 convolution with 64 output filters on a 256x256 image:
model = Sequential()
model.add(Dense(2000, input_shape=(1100,),activation='relu',kernel_regularizer=regularizers.l2(1e-3),
                activity_regularizer=regularizers.l2(1e-3)))

# now model.output_shape == (None, 64, 256, 256)

# add a 3x3 convolution on top, with 32 output filters:
model.add(Dense(1000, input_shape=(2000,)))
# now model.output_shape == (None, 32, 256, 256)
model.add(Dense(500, input_shape=(1000,)))
model.add(Dense(19, input_shape=(500,)))
###
adam = Adam(lr=1e-3)
model.compile(loss='mse',optimizer=adam)
print("We finish building the model")

plot_model(model, to_file='model1.png', show_shapes=True)

We finish building the model


In [7]:
MicroArray,Labels = read_data(mode="DiseaseStage",th=20)

Loading Previous Data!


In [8]:
print MicroArray.shape
print np.unique(Labels).shape

(3053, 22283)
(43,)


In [3]:
## shuffle data and divide it into train & set
from util import *

MicroArray_train,MicroLabels_train,MicroArray_test,MicroLabels_test = division(MicroArray,Labels)

MicroLabels_train = MicroLabels_train.reshape([MicroLabels_train.shape[0]])
MicroLabels_test = MicroLabels_test.reshape([MicroLabels_test.shape[0]])

print MicroArray_train.shape
print MicroArray_test.shape
print MicroLabels_train.shape
print MicroLabels_test.shape

(2458, 22283)
(595, 22283)
(2458,)
(595,)


In [4]:
import liblinearutil as linear_classifier

tic = time.time()
prob = linear_classifier.problem(MicroLabels_train,MicroArray_train)

param = linear_classifier.parameter('-s 0 -c 20 -B 1')
model = linear_classifier.train(prob,param)

p_label,p_acc,p_val = linear_classifier.predict(MicroLabels_test,MicroArray_test,model)
ACC,MSE,SCC = linear_classifier.evaluations(MicroLabels_test,p_label)
toc = time.time()

print "total time:",toc-tic
print ACC

Accuracy = 87.0588% (518/595) (classification)
total time: 390.692141056
87.0588235294


# Try PCA

In [ ]:
# sing_cumsum = np.cumsum(sing_vals)
# sing_ratio = sing_cumsum/np.sum(sing_vals)
# plt.plot(sing_ratio)
# for cut in range(3000,3600,100):
#     print cut,sing_ratio[cut]
#     plt.plot(cut,sing_ratio[cut],"ro")
# plt.show()

In [ ]:
# cut = 3000

# U_cut = U[:,0:cut]
# sing_cut = sing_vals[0:cut]
# V_cut = V[0:cut,:]
# print U_cut.shape,sing_cut.shape,V_cut.shape

# ProjArray = U_cut.dot(np.diag(sing_cut))
# print ProjArray.shape

In [5]:
ProjArray = pca(MicroArray,0.95)

2322


In [6]:
## shuffle data and divide it into 5 parts
from util import *

ProjLabels = Labels
ProjArray_train,ProjLabels_train,ProjArray_test,ProjLabels_test = division(ProjArray,ProjLabels)
ProjLabels_train = ProjLabels_train.reshape([ProjLabels_train.shape[0]])
ProjLabels_test = ProjLabels_test.reshape([ProjLabels_test.shape[0]])

print ProjArray_train.shape
print ProjArray_test.shape
print ProjLabels_train.shape
print ProjLabels_test.shape

(2458, 2322)
(595, 2322)
(2458,)
(595,)


In [7]:
import liblinearutil as linear_classifier

tic = time.time()
prob = linear_classifier.problem(ProjLabels_train,ProjArray_train)
param = linear_classifier.parameter('-s 0 -c 20 -B 1')
model = linear_classifier.train(prob,param)

p_label,p_acc,p_val = linear_classifier.predict(ProjLabels_test,ProjArray_test,model)
ACC,MSE,SCC = linear_classifier.evaluations(ProjLabels_test,p_label)
toc = time.time()

print "total time:",toc-tic
print ACC

Accuracy = 87.7311% (522/595) (classification)
total time: 268.804051161
87.731092437


# low rank

In [3]:
from rpca import *
A = rpcaADMM(MicroArray)
LowRank = A['X3_admm']


iter	    r norm	   eps pri	    s norm	  eps dual	 objective
   1	16985.1815	  286.0049	28457.6289	  280.8639	155164890.16
  10	 6706.6550	  424.3059	 4182.1618	  270.5618	528273850.53
  20	 1838.8322	  419.0288	  931.2457	  171.9436	425203466.44
  30	  722.4489	  406.1955	  377.6303	  172.2705	395442242.81


In [ ]:
# singualr values already stored in descending order
LR_U, LR_sing_vals, LR_V = np.linalg.svd(LowRank, full_matrices=False)

In [ ]:
cut = 100
LR_U_cut = LR_U[:,0:cut]
LR_sing_cut = LR_sing_vals[0:cut]
LR_V_cut = LR_V[0:cut,:]
LR_ProjArray = MicroArray.dot(LR_V_cut.T)
#LR_ProjArray = LR_U_cut.dot(np.diag(LR_sing_cut))

print LR_ProjArray.shape

In [ ]:
## shuffle data and divide it into 5 parts
                                                                                                            from util import *
    
                    Labels                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                            LR_ProjLabels = Labels
LR_ProjArray_train,LR_ProjLabels_train,LR_ProjArray_test,LR_ProjLabels_test = division(LR_ProjArray,LR_ProjLabels)
LR_ProjLabels_train = LR_ProjLabels_train.reshape([LR_ProjLabels_train.shape[0]])
LR_ProjLabels_test = LR_ProjLabels_test.reshape([LR_ProjLabels_test.shape[0]])

print LR_ProjArray_train.shape
print LR_ProjArray_test.shape
print LR_ProjLabels_train.shape
print LR_ProjLabels_test.shape

In [ ]:
import liblinearutil as linear_classifier

tic = time.time()
prob = linear_classifier.problem(LR_ProjLabels_train,LR_ProjArray_train)
param = linear_classifier.parameter('-s 0 -c 20 -B 1')
model = linear_classifier.train(prob,param)

p_label,p_acc,p_val = linear_classifier.predict(LR_ProjLabels_test,LR_ProjArray_test,model)
ACC,MSE,SCC = linear_classifier.evaluations(LR_ProjLabels_test,p_label)
toc = time.time()

print "total time:",toc-tic
print ACC